# Understanding and Building Contoso Creative 

This Jupyter notebook is an environment that allows you to run Python code in a clear way. 
To begin, first click the `Select Kernel` button on the top right of the screen this should open a menu above that prompts you to choose a kernel. Select `Python evironments` and then choose `Python 3.11.9`. 

Once this has been done, go through each section of this notebook and run the cells when instructed to by clicking the `play` button on the left side of the cell with the code. 

## Learning Outcomes

As a reminder we will focus on 4 learning outcomes:

1. Understanding agents and prompt engineering with Prompty 

2. Utilizing Prompty tracing for debugging and observabilty

3. Building and running Contoso Creative Writer

4. Setting up automated evaluations with Github Actions 

Let's start with the first one. 

## 1. Understanding agents and prompt engineering with Prompty  

### What are AI Agents
As you should have heard from your instructor Contoso Creative Writer is an Agentic Application. 

In Artificial Intelligence, an agent is a computer program or system that is designed to:

 **percieve it's environment, make decisions, and take actions to achieve a specific goal or set of goals**. 

For Contoso Creative Writer the overall goal is to create an app that will help the marketing team at a fictional company called Contoso Outdooors write well researched, product specific articles. 

This goal is achieved by 4 different agents that percieve their environments and take actions to complete 4 sub-goals that help achieve the final goal. 

**Contoso Creative Writer is made up of 4 agents**:

<img src="agents.png" alt="Agents in contoso creative writer" width="900" height="380">

### Building an Agent with Prompty and Azure OpenAI 

To understand how an agent is built we will be focusing on the `Researcher Agent`. 

This agent searches for relevant information online using Bing Search and Azure OpenAI with GPT 3.5 Turbo model. 

To illustrate what it does click the play button below and change the topic to test it out with a topic of your choice. 

In [ ]:
import sys
import os

# Add the path to sys.path
sys.path.append(os.path.abspath('../../src/api/agents/researcher'))

from researcher import research

instructions = "Can you find the latest camping trends and what folks are doing in the winter?"

research(instructions=instructions)

## 2. Utilizing Prompty tracing for debugging and observabilty

## 3. Building and running Contoso Creative Writer 

## 4. Setting up automated evaluations with Github Actions 